# Lab 1: Intro to TensorFlow and Music Generation with RNNs
# Part 1: Intro to TensorFlow

TensorFlow is a software library extensively used in machine learning. Here we'll learn how computations are represented and how to define simple neural networks in TensorFlow.

In [0]:
!pip install tf-nightly-gpu-2.0-preview

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [0]:
is_correct_tf_version = '1.13.' in tf.__version__
assert is_correct_tf_version, "Wrong tensorflow version {} installed".format(tf.__version__)

is_eager_enabled = tf.executing_eagerly()
assert is_eager_enabled,      "Tensorflow eager mode is not enabled"

## 1.1 The Computation Graph

TODO: FIX. Include explanation of Keras API and Eager in this part?
TensorFlow is called TensorFlow because it handles the flow (node/mathematical operation) of Tensors (data). You can think of a Tensor as a multidimensional array. In TensorFlow, computations are represented as graphs. TensorFlow programs are usually structured into a phase that assembles a graph, and a phase that uses a session to execute operations in the graph. In TensorFlow we define the computational graph with Tensors and mathematical operations to create a system for machine learning and deep learning.

We can think of a computation graph as a series of math operations that occur in some order. First let's look at a simple example:

![alt text](img/add-graph.png "Computation Graph")


In [167]:
# create nodes in a graph
a = tf.constant(15, name="a")
b = tf.constant(61, name="b")

# add them
c = tf.add(a,b, name="c")
print(c)

tf.Tensor(76, shape=(), dtype=int32)


Notice that the output is still an abstract Tensor -- we've just created a computation graph consisting of operations. To actually get our result, we have to define and execute a `Model` using the Keras API. We'll do that next.
TODO: FIX

### 1.1.1 Building and Executing the Computation Graph

Consider the following computation graph:

![alt text](img/computation-graph.png "Computation Graph")

This graph takes 2 inputs, `a, b` and computes an output `e`. Each node in the graph is an operation that takes some input, does some computation, and passes its output to another node.

Let's first construct this computation graph in TensorFlow:

In [0]:
# construct the computation graph
def graph(a,b):
  '''TODO: Define the operation for c, d, e (use tf.add, tf.subtract, tf.multiply).'''
  c = tf.add(a, b)
  var = tf.Variable(1.0)
  d = tf.subtract(b, var)
  e = tf.multiply(c, d)
  return e

Now we're going to create a callable function to execute our computation graph:
TODO: FIX

Let's go through the execution above step-by-step. First, we used `tf.keras.Input` to define the inputs, and then defined the operations for `c`, `d`, and `e`. We then created a callable `Model` to execute our computation graph, defined our input data, and executed the computation graph to provide the output values given the input data. 
TODO: FIX

## 1.2 Neural Networks in TensorFlow
We can define neural networks in TensorFlow using computation graphs. Here is an example of a very simple neural network (just 1 perceptron):
TODO: FIX the graph so that the yellow uses tf.keras.Input rather than tf.placeholder; FIX the description to reflect the use of Sequential model in Keras, use of Dense layer

![alt text](img/computation-graph-2.png "Computation Graph")

This graph takes an input, (x) and computes an output (out). It does so how we learned in lecture today: `out = sigmoid(W*x+b)`.

We could build this computation graph in TensorFlow in the following way:

In [0]:
def our_dense_layer(x, n_in, n_out):
  w = tf.Variable(tf.ones((n_in, n_out)))
  b = tf.Variable(tf.zeros((1, n_out)))

  z = tf.matmul(x,w) + b
  
  output = tf.sigmoid(z)
  return output

In [194]:
x_input = tf.constant([[1,2.]], shape=(1,2))
print our_dense_layer(x_input, n_in=2, n_out=3)

tf.Tensor([[0.95257413 0.95257413 0.95257413]], shape=(1, 3), dtype=float32)


Now we will use the `Sequential` model from Keras to define a model with a single fully connected layer. 
TODO: FIX

In [202]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# define the number of inputs and outputs
n_input_nodes = 2
n_output_nodes = 3

# first define the model 
model = Sequential()

# define a dense (fully connected) layer to compute z 
dense_layer = Dense(n_output_nodes, input_shape=(n_input_nodes,),activation='sigmoid')

# add the dense layer to the model
model.add(dense_layer)

# general form for the input
x = tf.keras.layers.Input(shape=(n_input_nodes,))

# test with example input
x_input = tf.constant([[1,2.]], shape=(1,2))
print model(x_input)





tf.Tensor([[0.9216483  0.49162707 0.4871195 ]], shape=(1, 3), dtype=float32)


## 1.3 Eager execution

The 6.S191 team is **Eager** to show you one of the coolest recent developments in TensorFlow: Eager execution. Eager is an experimental interface to TensorFlow that provides an imperative programming style. When you enable Eager execution, TensorFlow operations execute immediately as they're called from Python. That means you do not execute a pre-constructed graph with `Session.run()`. This allows for fast debugging and a more intuitive way to get started with TensorFlow.

First, we must enable Eager execution. When we do this, operations will execute and return their values immediately. Some things to note:

- We will need to restart the Python kernel since we have already used TensorFlow in graph mode. 
- We enable eager at program startup using: `tfe.enable_eager_execution()`.
- Once we enable Eager with `tfe.enable_eager_execution()`, it cannot be turned off. To get back to graph mode, start a new Python session.

### 1.3.1 How is Eager Different?
Before we get started with Eager, let's see what happens when we define a simple operation in graph mode:

In [0]:
print(tf.add(1, 2))

This tells us that we're just building a computation graph with the above operation, and not actually executing anything. Let's see how Eager is different. We restart the Python kernel and start Eager execution. 
**This command will cause your kernel to die but this is okay since we are restarting.**

In [0]:
import os
os._exit(00)

In [0]:
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
import tensorflow as tf

Let's run the same operation as before -- adding 1 and 2 -- in Eager mode:

In [0]:
print(tf.add(1, 2))

Cool! We just defined and executed an operation in TensorFlow immediately as it was called.

### 1.3.2 Automatic Differentiation
Automatic differentiation is very useful when implementing many machine learning algorithms such as backpropagation for training neural networks. For this purpose, TensorFlow Eager execution provides an [autograd](https://github.com/HIPS/autograd)	style API for automatic differentiation.

In [0]:
def f(x):
    # f(x) = x^2 + 3
    return tf.multiply(x, x) + 3

print( "f(4) = %.2f" % f(4.) )

# First order derivative
df = tfe.gradients_function(f) # tfe == eager mode
print( "df(4) = %.2f" % df(4.)[0] )

# Second order derivative
'''TODO: fill in the expression for the second order derivative using Eager mode gradients'''
d2f = # TODO
print( "d2f(4) = %.2f" % d2f(4.)[0] )

### 1.3.3 Dynamic Models

Dynamic models can be built with Python flow control. Here's an example of the [Collatz conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture) using TensorFlow’s arithmetic operations. Such dynamic behavior is not possible in standard TensorFlow (up to v1.4):

In [0]:
a = tf.constant(12)
counter = 0
while not tf.equal(a, 1):
  if tf.equal(a % 2, 0):
    a = a / 2
  else:
    a = 3 * a + 1
  print(a)